# Spark SQL: Stack Exchange SciFi community

## Introduction

Stack Exchange is a [network of Q&A communities](http://stackexchange.com/sites#), covering a wide range of topics. Its first instance, Stack Overflow is today one of the most prolific examples of collaborative knowledge creation in Internet. [Science Fiction and Fantasy (SciFi)](http://scifi.stackexchange.com/) in one of such communities, focused on questions on science fiction and fantasy.

Stack Exchange also follows an open data approach regarding the activity records collected from all its sites. The [Data Explorer Service](https://data.stackexchange.com/) allows users to query activity datasets from any community interactively. In addition, the organization [regularly publishes](https://archive.org/details/stackexchange) a full dump (torrent of files) including the whole set of activity log records from all sites, for offline data analysis.

In this example, we load and query a excerpt of one of such files in XML format to illustrate Spark SQL features. To read XML files in Spark we make use of an external library, the [XML Data Source for Apache Spark](https://github.com/databricks/spark-xml) developed by Databricks.

## Preparation

Furthermore, we must launch `pyspark` with the **`--packages com.databricks:spark-xml_2.12:0.14.0`** option, so that Jupyter loads this dependency when we open the notebook.

**IMPORTANT**: Please note that, **depending on your specific version of Spark**, you must choose a compatible version of `spark-xml` that also matches the Scala version used to compile it. **Please, refer to the [summary table available on the project page in GitHub](https://github.com/databricks/spark-xml#requirements) to select the appropriate combination for your platform**.

In [1]:
# Load external packages programatically
# Here, we assume that you use Spark 3.2.1 or later (compiled against Scala 2.12)
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

packages = "com.databricks:spark-xml_2.12:0.16.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

In [2]:
os.environ["JAVA_HOME"]

'/usr/lib/jvm/java-17-openjdk-amd64'

In [3]:
os.environ["PYSPARK_SUBMIT_ARGS"]

'--packages com.databricks:spark-xml_2.12:0.16.0 pyspark-shell'

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("StackExchange SciFi")
    .getOrCreate() )
sc = spark.sparkContext

23/03/25 12:36:48 WARN Utils: Your hostname, helium resolves to a loopback address: 127.0.1.1; using 10.6.36.17 instead (on interface wlo1)
23/03/25 12:36:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/jfelipe/miniconda3/envs/pyspark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jfelipe/.ivy2/cache
The jars for the packages stored in: /home/jfelipe/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3b353100-e5cf-4dbf-9079-d5bc40d8fb73;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.16.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 237ms :: artifacts dl 12ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.16.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	------

23/03/25 12:36:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
from pyspark.sql.types import *
from pyspark.sql import functions
# Define data scheme
# Each XML element in the tree is identified by its tag
# We can use @attr_name to refer to an attribute of an element
customSchema = StructType([
                    StructField("AccountId", LongType(), True), 
                    StructField("Age", ByteType(), True),
                    StructField("CreationDate", TimestampType(), True),
                    StructField("DisplayName", StringType(), True),
                    StructField("DownVotes", IntegerType(), True),
                    StructField("Id", LongType(), True),
                    StructField("LastAccessDate", TimestampType(), True),
                    StructField("Location", StringType(), True),
                    StructField("ProfileImageUrl", StringType(), True),
                    StructField("Reputation", IntegerType(), True),
                    StructField("UpVotes", IntegerType(), True),
                    StructField("Views", IntegerType(), True),
                    StructField("WebsiteUrl", StringType(), True)
                          ])

In [7]:
# Direct data loading from XML file
df = (spark.read.format('xml')
                     .options(rowTag='row')
                     .load('../data/Users_conv2.xml',
                           schema = customSchema)
     )

In [8]:
# Total number of users in this excerpt from complete SciFi dump
df.count()

29831

In [9]:
# Print DataFrame content scheme
df.printSchema()

root
 |-- AccountId: long (nullable = true)
 |-- Age: byte (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- Id: long (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- WebsiteUrl: string (nullable = true)



In [10]:
df.head(6)

[Row(AccountId=-1, Age=None, CreationDate=datetime.datetime(2011, 1, 11, 20, 19, 36, 483000), DisplayName='Community', DownVotes=3953, Id=-1, LastAccessDate=datetime.datetime(2011, 1, 11, 20, 19, 36, 483000), Location='on the server farm', ProfileImageUrl=None, Reputation=1, UpVotes=2587, Views=0, WebsiteUrl='http://meta.stackexchange.com/'),
 Row(AccountId=2, Age=38, CreationDate=datetime.datetime(2011, 1, 11, 20, 50, 40, 620000), DisplayName='Geoff Dalgas', DownVotes=0, Id=2, LastAccessDate=datetime.datetime(2015, 2, 5, 1, 3, 28, 30000), Location='Corvallis, OR', ProfileImageUrl=None, Reputation=101, UpVotes=1, Views=21, WebsiteUrl='http://stackoverflow.com'),
 Row(AccountId=7598, Age=30, CreationDate=datetime.datetime(2011, 1, 11, 20, 55, 41, 460000), DisplayName='Nick Craver', DownVotes=0, Id=3, LastAccessDate=datetime.datetime(2015, 3, 1, 14, 49, 49, 173000), Location='Winston-Salem, NC', ProfileImageUrl='http://i.stack.imgur.com/nGCYr.jpg', Reputation=101, UpVotes=3, Views=10, We

In [16]:
spark

## SQL Queries

In [12]:
# Register temp table for SQL queries (not required in newer Spark versions)
# df.registerTempTable('scifi')
df.createOrReplaceTempView('scifi')

In [17]:
# Find all users (Id, DisplayName) based in San Francisco (using simple regexp)
loc_SFco = spark.sql("""
SELECT Id, DisplayName
FROM scifi
WHERE (Id >= 300)
""")
loc_SFco.show(10)

+---+--------------+
| Id|   DisplayName|
+---+--------------+
|300|       BozoJoe|
|301|  Neil Trodden|
|302|        Uberto|
|304|   Nick Haslam|
|305|           rmx|
|307|       Nerevar|
|309|      Uwe Keim|
|310|Wouter Lievens|
|311| Tim Schmelter|
|312|           nik|
+---+--------------+
only showing top 10 rows



In [15]:
loc_SFco.count()

245

In [21]:
# Find all users (Id, DisplayName) based in San Francisco 
# (using simple regexp)
loc_US = spark.sql("""
SELECT Id, DisplayName, Location
FROM scifi
WHERE Location LIKE '%USA%' OR Location LIKE '%US%' OR Location LIKE 'USA%'
                            OR Location LIKE 'US%'
""")
loc_US.show(30)

+----+------------------+--------------------+
|  Id|       DisplayName|            Location|
+----+------------------+--------------------+
| 250|     John Saunders|    Phoenix, AZ, USA|
| 558|           JYelton|           Utah, USA|
|1205|          Peter K.|             CT, USA|
|1293|       Dan Herbert|San Francisco Bay...|
|1397|         gallamine|             NC, USA|
|1653|          zemoxian|Silver Spring, MD...|
|1693|             Tango|   Richmond, VA, USA|
|2033|        Thom Blake|  New Haven, CT, USA|
|2529|           AruniRC|                 USA|
|2545|             gef05| North Carolina, USA|
|2615|             Alger|             NJ, USA|
|3584|          Unsigned|     West Coast, USA|
|3603|         Thomas W.| Pittsburgh, PA, USA|
|3757|   Brian Wigginton|     Austin, TX, USA|
|3850|               YHZ|                 USA|
|4102|         ryuuyasha|                  US|
|4133|               JNK|             CT, USA|
|4407|    Adrian Cornish|                 USA|
|4524|Franck 

In [22]:
loc_US.count()

129

In [23]:
# Find the min and max age of all users who created a new account on March 2014 or later
users_after_210403 = spark.sql("""
SELECT MIN(Age) AS Min_Age, MAX(Age) AS Max_Age
FROM scifi
WHERE CreationDate > '2014-03-01'
""")
users_after_210403.show()

+-------+-------+
|Min_Age|Max_Age|
+-------+-------+
|     13|     95|
+-------+-------+



In [24]:
# Show the user with the max. reputation in this excerpt
max_reputation = spark.sql("""
SELECT MAX(Reputation) FROM scifi
""")
max_reputation.show()

+---------------+
|max(Reputation)|
+---------------+
|         148259|
+---------------+



In [25]:
# Display metadata for that user with max. reputation
user_max_reputation = spark.sql("""
SELECT Id, DisplayName, UpVotes, DownVotes FROM scifi
WHERE Reputation = 148259
""")
user_max_reputation.show()

+---+-----------+-------+---------+
| Id|DisplayName|UpVotes|DownVotes|
+---+-----------+-------+---------+
|976|        DVK|   5503|     1806|
+---+-----------+-------+---------+



In [26]:
user_max_reputation = spark.sql("""
SELECT Id, DisplayName, UpVotes, DownVotes FROM scifi
WHERE Reputation = (SELECT MAX(Reputation) FROM scifi)
""")
user_max_reputation.show()

+---+-----------+-------+---------+
| Id|DisplayName|UpVotes|DownVotes|
+---+-----------+-------+---------+
|976|        DVK|   5503|     1806|
+---+-----------+-------+---------+



## The End

In [8]:
# Remember to stop SparkContext before shutting down this notebook
spark.stop()